# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 13 2022 10:36AM,241736,10,PRF-33125,Bio-PRF,Released
1,Jul 13 2022 10:35AM,241753,10,8269443,Eywa Pharma Inc.,Released
2,Jul 13 2022 10:35AM,241753,10,8269445,Eywa Pharma Inc.,Released
3,Jul 13 2022 10:35AM,241753,10,8269447,Eywa Pharma Inc.,Released
4,Jul 13 2022 10:35AM,241753,10,8269449,Eywa Pharma Inc.,Released
5,Jul 13 2022 10:35AM,241753,10,8269450,Eywa Pharma Inc.,Released
6,Jul 13 2022 10:35AM,241753,10,8269451,Eywa Pharma Inc.,Released
7,Jul 13 2022 10:35AM,241753,10,8269453,Eywa Pharma Inc.,Released
8,Jul 13 2022 10:35AM,241753,10,8269458,Eywa Pharma Inc.,Released
9,Jul 13 2022 10:35AM,241753,10,8269460,Eywa Pharma Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
45,241749,Released,1
46,241750,Released,14
47,241752,Released,1
48,241753,Released,18
49,241755,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
241749,NaN,NaN,1.0
241750,NaN,NaN,14.0
241752,NaN,NaN,1.0
241753,NaN,NaN,18.0
241755,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241604,0.0,0.0,22.0
241644,0.0,0.0,1.0
241648,1.0,0.0,4.0
241654,0.0,0.0,1.0
241657,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241604,0,0,22
241644,0,0,1
241648,1,0,4
241654,0,0,1
241657,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241604,0,0,22
1,241644,0,0,1
2,241648,1,0,4
3,241654,0,0,1
4,241657,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241604,,,22
1,241644,,,1
2,241648,1,,4
3,241654,,,1
4,241657,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 13 2022 10:36AM,241736,10,Bio-PRF
1,Jul 13 2022 10:35AM,241753,10,Eywa Pharma Inc.
19,Jul 13 2022 10:34AM,241755,16,Sartorius Bioprocess Solutions
20,Jul 13 2022 10:29AM,241752,10,ISDIN Corporation
21,Jul 13 2022 10:23AM,241750,20,"ACI Healthcare USA, Inc."
35,Jul 13 2022 10:05AM,241749,12,LF of America Corp.
36,Jul 13 2022 10:03AM,241748,20,"ACI Healthcare USA, Inc."
37,Jul 13 2022 9:46AM,241747,22,"NBTY Global, Inc."
38,Jul 13 2022 9:42AM,241746,22,"NBTY Global, Inc."
39,Jul 13 2022 9:36AM,241744,12,Hush Hush


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 13 2022 10:36AM,241736,10,Bio-PRF,,,1
1,Jul 13 2022 10:35AM,241753,10,Eywa Pharma Inc.,,,18
2,Jul 13 2022 10:34AM,241755,16,Sartorius Bioprocess Solutions,,,1
3,Jul 13 2022 10:29AM,241752,10,ISDIN Corporation,,,1
4,Jul 13 2022 10:23AM,241750,20,"ACI Healthcare USA, Inc.",,,14
5,Jul 13 2022 10:05AM,241749,12,LF of America Corp.,,,1
6,Jul 13 2022 10:03AM,241748,20,"ACI Healthcare USA, Inc.",,,1
7,Jul 13 2022 9:46AM,241747,22,"NBTY Global, Inc.",,,1
8,Jul 13 2022 9:42AM,241746,22,"NBTY Global, Inc.",,,1
9,Jul 13 2022 9:36AM,241744,12,Hush Hush,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 13 2022 10:36AM,241736,10,Bio-PRF,1,,
1,Jul 13 2022 10:35AM,241753,10,Eywa Pharma Inc.,18,,
2,Jul 13 2022 10:34AM,241755,16,Sartorius Bioprocess Solutions,1,,
3,Jul 13 2022 10:29AM,241752,10,ISDIN Corporation,1,,
4,Jul 13 2022 10:23AM,241750,20,"ACI Healthcare USA, Inc.",14,,
5,Jul 13 2022 10:05AM,241749,12,LF of America Corp.,1,,
6,Jul 13 2022 10:03AM,241748,20,"ACI Healthcare USA, Inc.",1,,
7,Jul 13 2022 9:46AM,241747,22,"NBTY Global, Inc.",1,,
8,Jul 13 2022 9:42AM,241746,22,"NBTY Global, Inc.",1,,
9,Jul 13 2022 9:36AM,241744,12,Hush Hush,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 13 2022 10:36AM,241736,10,Bio-PRF,1,,
1,Jul 13 2022 10:35AM,241753,10,Eywa Pharma Inc.,18,,
2,Jul 13 2022 10:34AM,241755,16,Sartorius Bioprocess Solutions,1,,
3,Jul 13 2022 10:29AM,241752,10,ISDIN Corporation,1,,
4,Jul 13 2022 10:23AM,241750,20,"ACI Healthcare USA, Inc.",14,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 13 2022 10:36AM,241736,10,Bio-PRF,1.0,NaN,NaN
1,Jul 13 2022 10:35AM,241753,10,Eywa Pharma Inc.,18.0,NaN,NaN
2,Jul 13 2022 10:34AM,241755,16,Sartorius Bioprocess Solutions,1.0,NaN,NaN
3,Jul 13 2022 10:29AM,241752,10,ISDIN Corporation,1.0,NaN,NaN
4,Jul 13 2022 10:23AM,241750,20,"ACI Healthcare USA, Inc.",14.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 13 2022 10:36AM,241736,10,Bio-PRF,1.0,0.0,0.0
1,Jul 13 2022 10:35AM,241753,10,Eywa Pharma Inc.,18.0,0.0,0.0
2,Jul 13 2022 10:34AM,241755,16,Sartorius Bioprocess Solutions,1.0,0.0,0.0
3,Jul 13 2022 10:29AM,241752,10,ISDIN Corporation,1.0,0.0,0.0
4,Jul 13 2022 10:23AM,241750,20,"ACI Healthcare USA, Inc.",14.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4592473,167.0,54.0,1.0
12,966978,31.0,0.0,0.0
15,483313,42.0,0.0,0.0
16,966927,22.0,2.0,0.0
19,966837,9.0,21.0,4.0
20,1450328,64.0,7.0,0.0
21,966759,4.0,0.0,0.0
22,483493,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4592473,167.0,54.0,1.0
1,12,966978,31.0,0.0,0.0
2,15,483313,42.0,0.0,0.0
3,16,966927,22.0,2.0,0.0
4,19,966837,9.0,21.0,4.0
5,20,1450328,64.0,7.0,0.0
6,21,966759,4.0,0.0,0.0
7,22,483493,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,167.0,54.0,1.0
1,12,31.0,0.0,0.0
2,15,42.0,0.0,0.0
3,16,22.0,2.0,0.0
4,19,9.0,21.0,4.0
5,20,64.0,7.0,0.0
6,21,4.0,0.0,0.0
7,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,167.0
1,12,Released,31.0
2,15,Released,42.0
3,16,Released,22.0
4,19,Released,9.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21,22
Status,,,,,,,,
Completed,1.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0
Executing,54.0,0.0,0.0,2.0,21.0,7.0,0.0,0.0
Released,167.0,31.0,42.0,22.0,9.0,64.0,4.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21,22
0,Completed,1.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0
1,Executing,54.0,0.0,0.0,2.0,21.0,7.0,0.0,0.0
2,Released,167.0,31.0,42.0,22.0,9.0,64.0,4.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21,22
0,Completed,1.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0
1,Executing,54.0,0.0,0.0,2.0,21.0,7.0,0.0,0.0
2,Released,167.0,31.0,42.0,22.0,9.0,64.0,4.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()